# 솔루션 생성 및 평가 <a class="anchor" id="top"></a>

이 노트북에서는 Amazon Personalize를 사용하여 여러 모델을 훈련합니다. 구체적으로는 다음과 같습니다.

1. 사용자 개인화 - 특정 사용자와 가장 연관성이 있는 항목은 무엇인가?
1. 유사 항목 - 주어진 항목의 유사 항목은 무엇인가?
1. 개인화된 순위 - 사용자와 항목 모음이 주어진 경우, 가장 연관성이 높은 순서는 무엇인가?

## 개요

1. [소개](#intro)
1. [솔루션 생성](#solutions)
1. [솔루션 평가](#eval)
1. [평가 지표 사용](#use)
1. [유용한 변수 저장](#vars)

## 소개 <a class="anchor" id="intro"></a>

다시 말하지만, 대부분의 경우 Amazon Personalize(레시피라고 함)의 알고리즘은 여기서 설명하는 다양한 작업을 처리합니다.

1. **사용자 개인화** - 모든 HRNN 워크플로/사용자 개인화 요구 사항을 지원하는 새로운 릴리스로, 여기서는 이 릴리스를 사용합니다.
1. **HRNN 및 HRNN 메타데이터** - 이전 사용자 상호 작용에 따라 항목을 추천합니다.
1. **HRNN 콜드 스타트** - 아직 상호 작용 데이터가 확보되지 않은 새 항목을 추천합니다.
1. **개인별 순위** - 항목 모음을 선택한 다음 HRNN과 유사한 접근 방식을 사용하여 항목 순서를 지정합니다.
1. **SIMS(유사 항목)** - 특정 항목이 주어지면 사용자가 상호 작용하는 다른 항목도 추천합니다.
1. **인기 카운트** - HRNN 또는 HRNN 메타데이터에서 적절한 정보를 찾을 수 없는 경우 가장 인기 있는 항목을 추천합니다. 이 항목은 기본적으로 반환됩니다.

사용 사례에 관계없이, 모든 알고리즘은 3가지 핵심 속성으로 정의된 사용자-항목-상호 작용 데이터에 대한 학습 기반을 공유합니다.

1. **UserID** - 상호 작용한 사용자
1. **ItemID** - 사용자가 상호 작용한 항목
1. **타임스탬프** - 상호 작용이 발생한 시간

또한 다음과 같이 정의된 이벤트 유형과 이벤트 값도 지원합니다.

1. **이벤트 유형** - 이벤트의 범주형 레이블(찾아보기, 구매, 평점 등)입니다.
1. **이벤트 값** - 발생한 이벤트 유형에 해당하는 값입니다. 간단히 말해, 이벤트 유형에 대해 0과 1 사이의 정규화된 값을 찾는 것입니다. 예를 들어 3단계(클릭, 장바구니에 추가, 구매)로 거래를 완료하는 경우 각 단계마다 event_value는 각각 0.33, 0.66, 1.0이 됩니다.

이벤트 유형 및 이벤트 값 필드는 개인화 모델 훈련을 위해 전송된 데이터를 필터링하는 데 사용할 수 있는 추가 데이터입니다. 이 연습에서는 이벤트 유형이나 이벤트 값이 없습니다.

이 노트북을 실행하려면 이전 노트북 `01_Validating_and_Importing_User_Item_Interaction_Data` 및 `02_Validating_and_Importing_Item_Metadata.ipynb`를 실행하고 데이터 세트를 만든 후 상호 작용 및 항목 메타데이터 데이터를 Amazon Personalize로 가져와야 합니다. 이 노트북의 마지막에 변수 값 중 일부를 저장했습니다. 이제 이 값을 이 노트북에 로드해야 합니다.

In [1]:
%store -r

## 솔루션 생성 <a class="anchor" id="solutions"></a>
[맨 위로 이동](#top)

이 노트북에서는 다음 레시피로 솔루션을 만듭니다.

1. 사용자 개인화
1. SIMS
1. 개인별 순위

인기 카운트 레시피는 Amazon Personalize에서 사용할 수 있는 가장 간단한 솔루션이며 예비로만 사용해야 하므로 이 노트북에서도 다루지 않습니다.

이전 노트북과 마찬가지로 관련 패키지를 가져오는 것부터 시작하고 SDK를 사용하여 Amazon Personalize에 대한 연결을 설정합니다.

In [2]:
import time
from time import sleep
import json

import boto3

In [3]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

Amazon Personalize에서는 알고리즘의 특정 변형 버전을 레시피라고 합니다. 레시피마다 적용되는 상황이 다릅니다. 훈련된 모델을 솔루션이라고 하며, 각 솔루션은 모델을 훈련할 때 특정 데이터 볼륨과 관련된 여러 버전을 가질 수 있습니다.

먼저 지원되는 모든 레시피를 나열하겠습니다. 이렇게 하면 모델을 선택하고 모델을 구축하는 데 사용할 수 있습니다.

In [4]:
personalize.list_recipes()

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 8, 25, 16, 57, 0, 148000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 8, 25, 16, 57, 0, 148000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 8, 25, 16, 57, 0, 148000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   's

출력은 도입부에 언급된 모든 알고리즘의 JSON 표현일 뿐입니다.

다음으로 구체적인 레시피를 선택하여 그 레시피로 모델을 만듭니다.

### 사용자 개인화
사용자 개인화(aws-user-personalization) 레시피는 모든 USER_PERSONALIZATION 추천 시나리오에 최적화되어 있습니다. 이 레시피는 항목을 추천할 때 자동 항목 탐색을 사용합니다.

자동 탐색 기능을 통해 Amazon Personalize는 자동으로 여러 가지 항목 추천을 테스트하고, 사용자가 이러한 추천 항목과 상호 작용하는 방법을 학습하며, 더 나은 참여와 전환을 유도하는 항목에 대한 추천을 강화합니다. 이렇게 하면 카탈로그가 빠르게 변경되거나 뉴스 기사 또는 프로모션과 같은 새 항목이 새것인 상태일 때 사용자에게 더 연관성이 높은 경우, 항목 검색률과 참여율이 향상됩니다.

탐색할 양(상호 작용 데이터 또는 연관성이 적은 항목이 더 자주 추천되는 경우)과 활용할 양(추천이 우리가 알고 있는 내용 또는 연관성을 기반으로 하는 경우)을 균형 있게 조정할 수 있습니다. Amazon Personalize는 암시적인 사용자 피드백에 따라 향후 추천을 자동으로 조정합니다.

먼저 위의 레시피 목록에서 ARN을 찾아 레시피를 선택합니다.

In [5]:
user_personalization_recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

#### 솔루션 생성

먼저 레시피를 사용하여 솔루션을 만듭니다. 이 단계에서 데이터 세트 ARN을 제공하지만 모델은 아직 훈련되지 않았습니다. 따라서 훈련된 모델이 아니라 식별자로 생각해야 합니다.

In [6]:
user_personalization_create_solution_response = personalize.create_solution(
    name = "personalize-poc-userpersonalization",
    datasetGroupArn = dataset_group_arn,
    recipeArn = user_personalization_recipe_arn
)

user_personalization_solution_arn = user_personalization_create_solution_response['solutionArn']


In [7]:
print(json.dumps(user_personalization_solution_arn, indent=2))

"arn:aws:personalize:us-east-1:451584754870:solution/personalize-poc-userpersonalization"


#### 솔루션 버전 생성

솔루션을 찾은 후에는 모델 훈련을 완료하기 위해 버전을 만들어야 합니다. 훈련을 완료하는 데 25분 이상 걸릴 수 있으며, 이 예의 데이터 세트를 사용하여 이 레시피를 완료하는 데 평균 90분이 소요됩니다. 일반적으로 작업이 완료될 때까지 while 루프를 사용하여 폴링합니다. 하지만 이 작업은 다른 셀의 실행을 차단하며, 여기서 목표는 많은 모델을 만들고 신속하게 배포하는 것입니다. 따라서 노트북의 아래쪽에 있는 모든 솔루션에 대해 while 루프를 설정합니다. 또한 AWS 콘솔에서 진행 상황을 확인하는 방법에 대한 지침을 찾을 수 있습니다.

In [7]:
userpersonalization_create_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_solution_arn
)

In [8]:
userpersonalization_solution_version_arn = userpersonalization_create_solution_version_response['solutionVersionArn']
print(json.dumps(user_personalization_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-userpersonalization",
  "ResponseMetadata": {
    "RequestId": "11f510f4-f6da-4419-a0e4-567a504c5f9c",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 23:47:58 GMT",
      "x-amzn-requestid": "11f510f4-f6da-4419-a0e4-567a504c5f9c",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### SIMS


SIMS는 Amazon 내에서 추천 시스템에 사용되는 가장 오래된 알고리즘 중 하나입니다. 이 기능의 핵심 사용 사례는 항목이 하나 있고, 전체 사용자 기반에서 유사한 방식으로 상호 작용한 항목을 추천하려는 경우입니다. 즉, 결과가 사용자별로 개인화되지 않는 경우입니다. 이 경우 주로 인기 항목을 추천하게 되므로, 하이퍼파라미터를 조정하면서 결과에서 인기 항목을 줄일 수 있습니다.

이 예제 사용 사례의 경우 Movielens 데이터를 사용하여 특정 영화를 선택한다고 가정합니다. 그런 다음 SIMS를 사용하여 전체 사용자 기반의 상호 작용 행동을 기반으로 다른 영화를 추천할 수 있습니다. 결과는 사용자별로 개인화되지 않고 입력으로 선택한 영화에 따라 달라집니다.

지난 번처럼 레시피를 선택하는 것부터 시작합니다.

In [9]:
SIMS_recipe_arn = "arn:aws:personalize:::recipe/aws-sims"

#### 솔루션 생성

HRNN의 경우와 마찬가지로 먼저 솔루션을 만듭니다. 이 단계에서 데이터 세트 ARN을 제공하지만 모델은 아직 훈련되지 않았습니다. 따라서 훈련된 모델이 아니라 식별자로 생각해야 합니다.

In [10]:
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-sims",
  "ResponseMetadata": {
    "RequestId": "14c2cab0-6b36-4122-8f71-402acfa0c5e3",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 23:48:12 GMT",
      "x-amzn-requestid": "14c2cab0-6b36-4122-8f71-402acfa0c5e3",
      "content-length": "90",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### 솔루션 버전 생성

솔루션을 찾은 후에는 모델 훈련을 완료하기 위해 버전을 만들어야 합니다. 훈련을 완료하는 데 25분 이상 걸릴 수 있으며, 이 예의 데이터 세트를 사용하여 이 레시피를 완료하는 데 평균 35분이 소요됩니다. 일반적으로 작업이 완료될 때까지 while 루프를 사용하여 폴링합니다. 하지만 이 작업은 다른 셀의 실행을 차단하며, 여기서 목표는 많은 모델을 만들고 신속하게 배포하는 것입니다. 따라서 노트북의 아래쪽에 있는 모든 솔루션에 대해 while 루프를 설정합니다. 또한 AWS 콘솔에서 진행 상황을 확인하는 방법에 대한 지침을 찾을 수 있습니다.

In [11]:
sims_create_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)

In [12]:
sims_solution_version_arn = sims_create_solution_version_response['solutionVersionArn']
print(json.dumps(sims_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-sims/c1a33236",
  "ResponseMetadata": {
    "RequestId": "57e5bcee-b7cf-4eaf-8768-7f81c084232a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 23:48:15 GMT",
      "x-amzn-requestid": "57e5bcee-b7cf-4eaf-8768-7f81c084232a",
      "content-length": "106",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 개인별 순위

개인별 순위는 HRNN의 흥미로운 응용 사례 중 하나입니다. 이 알고리즘은 단순히 사용자에게 가장 유망한 항목을 추천하는 것이 아니라, 사용자는 물론 항목의 목록도 입력으로 받습니다. 그런 다음 사용자에게 가장 적합한 순서대로 항목이 다시 렌더링됩니다. 여기에 적합한 사용 사례는 필터를 생성할 항목 메타데이터가 없는 고유한 카테고리를 필터링하거나, 광범위한 컬렉션이 있는데 특정 사용자에게 더 나은 순서로 항목을 추천하기를 원하는 경우입니다.

이 예제의 경우, MovieLens 데이터를 사용하여 VOD 애플리케이션이 만화 영화 또는 특정 감독의 영화로 쉘프를 만드는 사용 사례를 떠올릴 수 있습니다. 이러한 목록 기반 제목 메타데이터가 있을 가능성이 높습니다. 이전 태그 기록을 기준으로 각 사용자의 영화 목록을 다시 정렬하려면 개인별 순위를 사용해야 합니다.

지난 번처럼 레시피를 선택하는 것부터 시작합니다.

In [13]:
rerank_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

#### 솔루션 생성

이전 솔루션과 마찬가지로 먼저 솔루션을 생성합니다. 이 단계에서 데이터 세트 ARN을 제공하지만 모델은 아직 훈련되지 않았습니다. 따라서 훈련된 모델이 아니라 식별자로 생각해야 합니다.

In [14]:
rerank_create_solution_response = personalize.create_solution(
    name = "personalize-poc-rerank",
    datasetGroupArn = dataset_group_arn,
    recipeArn = rerank_recipe_arn
)

rerank_solution_arn = rerank_create_solution_response['solutionArn']
print(json.dumps(rerank_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-rerank",
  "ResponseMetadata": {
    "RequestId": "932af10c-a961-41a1-bd53-079141b054c8",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 23:48:26 GMT",
      "x-amzn-requestid": "932af10c-a961-41a1-bd53-079141b054c8",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### 솔루션 버전 생성

솔루션을 찾은 후에는 모델 훈련을 완료하기 위해 버전을 만들어야 합니다. 훈련을 완료하는 데 25분 이상 걸릴 수 있으며, 이 예의 데이터 세트를 사용하여 이 레시피를 완료하는 데 평균 35분이 소요됩니다. 일반적으로 작업이 완료될 때까지 while 루프를 사용하여 폴링합니다. 하지만 이 작업은 다른 셀의 실행을 차단하며, 여기서 목표는 많은 모델을 만들고 신속하게 배포하는 것입니다. 따라서 노트북의 아래쪽에 있는 모든 솔루션에 대해 while 루프를 설정합니다. 또한 AWS 콘솔에서 진행 상황을 확인하는 방법에 대한 지침을 찾을 수 있습니다.

In [15]:
rerank_create_solution_version_response = personalize.create_solution_version(
    solutionArn = rerank_solution_arn
)

In [16]:
rerank_solution_version_arn = rerank_create_solution_version_response['solutionVersionArn']
print(json.dumps(rerank_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-rerank/eae37049",
  "ResponseMetadata": {
    "RequestId": "6799b3db-d3ef-4686-8a1f-05b22b42ab28",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 23:48:28 GMT",
      "x-amzn-requestid": "6799b3db-d3ef-4686-8a1f-05b22b42ab28",
      "content-length": "108",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 솔루션 생성 상태 보기

예고해 드린 대로, 콘솔에서 상태 업데이트를 보는 방법은 다음과 같습니다.

* 다른 브라우저 탭에서 이 노트북 인스턴스를 열 때 AWS 콘솔이 이미 열려 있어야 합니다.
* 해당 탭으로 전환하고 맨 위에서 `Personalize` 서비스를 검색한 다음 해당 서비스 페이지로 이동합니다.
* `View dataset groups`를 클릭합니다.
* 데이터 세트 그룹의 이름을 클릭합니다. 이름에 POC가 있을 가능성이 높습니다.
* `Solutions and recipes`를 클릭합니다.
* 이제 솔루션 버전 상태가 표시된 열을 포함하여, 위에서 생성한 모든 솔루션 목록이 표시됩니다. `Active` 상태가 되면 솔루션을 검토할 준비가 된 것입니다. 또한 배포할 수도 있습니다.

또는 아래의 셀을 실행하여 솔루션 버전 생성 상태를 추적합니다.

In [22]:
in_progress_solution_versions = [
    userpersonalization_solution_version_arn,
    sims_solution_version_arn,
    rerank_solution_version_arn
]

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

Build succeeded for arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-userpersonalization/236f000b
Build succeeded for arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-rerank/eae37049
At least one solution build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-sims/c1a33236


### 하이퍼파라미터 튜닝

Personalize는 솔루션을 만들 때 하이퍼파라미터 튜닝을 실행할 수 있는 옵션을 제공합니다. 하이퍼파라미터 튜닝을 수행하려면 추가 계산이 필요하기 때문에 이 기능은 기본적으로 비활성화되어 있습니다. 따라서 위에서 생성한 솔루션에서는 각 레시피에 하이퍼파라미터의 기본값을 사용합니다. 하이퍼파라미터 튜닝에 대한 자세한 내용은 [설명서](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html)를 참조하세요.

사용할 적절한 레시피를 결정하고 하이퍼파라미터 튜닝을 실행할 준비가 된 경우, 다음 코드는 SIMS 사용 사례를 예로 들어 하이퍼파라미터 튜닝 방법을 보여줍니다.

```python
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims-hpo",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn,
    performHPO=True
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))
```

특정 하이퍼파라미터에 사용할 값을 이미 알고 있는 경우, 솔루션을 만들 때 이 값을 설정할 수도 있습니다. 아래의 코드는 SIMS 레시피의 `popularity_discount_factor` 값을 설정하는 방법을 보여줍니다.

```python
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims-set-hp",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn,
    solutionConfig = {
        'algorithmHyperParameters': {
            'popularity_discount_factor': '0.7'
        }
    }
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))
```

## 솔루션 버전 평가 <a class="anchor" id="eval"></a>
[맨 위로 이동](#top)

한 시간이면 이 노트북의 모든 솔루션을 훈련할 수 있습니다. 훈련이 진행되는 동안 시간을 할애해 다양한 알고리즘(레시피)과 그 동작에 대해 자세히 읽어보시기 바랍니다. 또한 이 시간은 데이터를 시스템에 공급하는 방법과 예상되는 결과 유형에 대한 대안을 생각해보기에도 좋습니다.

솔루션 생성을 마쳤으면 다음 단계는 평가 지표를 가져오는 것입니다. Personalize는 훈련 데이터의 하위 집합을 기반으로 이러한 지표를 계산합니다. 아래 이미지는 Personalize를 사용하여 데이터를 분할하는 방법을 보여줍니다. 각각 10건의 상호 작용 기록이 있는 10명의 사용자가 있는 경우(원 하나는 상호 작용 하나를 나타냄) 상호 작용은 타임스탬프를 기준으로 가장 오래된 것부터 가장 최근의 것까지 순서가 매겨집니다. Personalize는 90%의 사용자(파란색 원)가 제공하는 모든 상호 작용 데이터를 사용하여 솔루션 버전을 훈련하고 나머지 10%는 평가에 사용합니다. 나머지 10%의 각 사용자에 대해 상호 작용 데이터(녹색 원)의 90%가 훈련된 모델에 대한 호출의 입력으로 사용됩니다. 나머지 10%의 데이터(주황색 원)는 모델에 의해 생성된 출력과 비교하여 평가 지표를 계산하는 데 사용됩니다.

![personalize metrics](static/imgs/personalize_metrics.png)

지표를 이해하려면 [설명서](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html)를 읽는 것이 좋지만, 편의를 위해 아래에 설명서의 일부를 복사해 두었습니다.

Personalize에서의 평가와 관련하여 다음 용어를 이해해야 합니다.

* 연관 추천은 특정 사용자에 대한 테스트 데이터의 값과 일치하는 추천을 말합니다.
* 순위는 추천 목록에서 추천 항목의 위치를 나타냅니다. 위치 1(목록 맨 위)은 사용자와 가장 연관성이 높은 것으로 간주됩니다.
* 쿼리는 GetRecommendations 호출에 해당하는 내부 정보를 말합니다.

Personalize에서 생성되는 지표는 다음과 같습니다.
* **coverage**: 훈련 데이터의 전체 고유 항목 수 중 모든 쿼리의 고유 추천 항목 비율(항목 및 상호 작용 데이터 세트 모두 포함)입니다.
* **mean_reciprocal_rank_at_25**: 모든 쿼리에 대한 상위 25개 추천 항목 중 첫 번째 관련 추천의 [평균 상호 순위](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)입니다. 이 지표는 순위가 가장 높은 단일 추천 항목을 파악하려는 경우 적합합니다.
* **normalized_discounted_cumulative_gain_at_K**: 절감 이득은 추천 목록에서 순위가 낮은 추천이 순위가 높은 추천보다 연관성이 적다고 가정합니다. 따라서 각 추천은 해당 위치에 따라 인수에 의해 절감됩니다(낮은 가중치가 부여됨). K에서 [누적 절감 이득](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)(DCG)을 산출하기 위해 상위 K개 추천의 각 연관 절감 추천을 함께 집계합니다. 정규화된 절감 누적 이득(NDCG)은 DCG를 이상적인 DCG로 나누어 NDCG가 0~1이 되도록 합니다. (이상적인 DCG란 상위 K 추천이 연관성에 따라 정렬하는 경우입니다.) Amazon Personalize는 1/log(1 + 위치)라는 가중치 인수를 사용합니다. 여기서 목록의 최상위는 위치 1입니다. 목록의 최상위에 있는 항목이 더 많은 주목을 받게 되므로, 이 지표는 목록 맨 위에 있는 연관 항목에 가중치를 부여합니다.
* **precision_at_K**: 상위 K개의 추천 중 연관 추천 수를 K로 나눈 값입니다. 이 지표는 연관 항목의 정확한 추천에 대해 가중치를 부여합니다.

이 노트북에서 생성된 각 솔루션의 평가 지표를 살펴보겠습니다. Movielens 데이터 세트의 품질로 인해 결과가 이 노트북의 텍스트에 설명된 결과와 다를 수 있습니다.

### 사용자 개인화 지표

먼저 사용자 개인화 솔루션 버전에 대한 평가 지표를 검색합니다.

In [23]:
user_personalization_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = userpersonalization_solution_version_arn
)

print(json.dumps(user_personalization_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-userpersonalization/236f000b",
  "metrics": {
    "coverage": 0.0791,
    "mean_reciprocal_rank_at_25": 0.269,
    "normalized_discounted_cumulative_gain_at_10": 0.2687,
    "normalized_discounted_cumulative_gain_at_25": 0.3321,
    "normalized_discounted_cumulative_gain_at_5": 0.2279,
    "precision_at_10": 0.0571,
    "precision_at_25": 0.0379,
    "precision_at_5": 0.075
  },
  "ResponseMetadata": {
    "RequestId": "fa6f0e49-b87d-48ce-ab27-80e61a1ab395",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 00:41:11 GMT",
      "x-amzn-requestid": "fa6f0e49-b87d-48ce-ab27-80e61a1ab395",
      "content-length": "417",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


위의 정규화된 절감 누적 이득은 5개 항목에서, 추천이 사용자 상호 작용 기록의 일부(훈련 및 검증의 보류 단계에서)일 확률이 (전체 38%, 소규모 22%)임을 있음을 나타냅니다. 추천 항목 중 약 13%가 고유하며, 상위 5개 추천 항목의 정밀도는 (전체 14%, 소규모 7.5%)에 불과합니다.

이는 분명 훌륭한 모델은 아니지만, Movielens는 등급에 기반한 명시적인 데이터 세트이기 때문에 이 상호 작용에 평점 데이터를 사용해야 한다는 것을 명심하세요. 또한 타임스탬프는 영화를 시청한 시점이 아니라 평점을 매긴 시점부터이므로, 그 순서가 시청자가 영화를 보는 순서와는 다릅니다.

### SIMS 지표

이제 SIMS 솔루션 버전에 대한 평가 지표를 검색합니다.

In [24]:
sims_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = sims_solution_version_arn
)

print(json.dumps(sims_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-sims/c1a33236",
  "metrics": {
    "coverage": 0.1861,
    "mean_reciprocal_rank_at_25": 0.1669,
    "normalized_discounted_cumulative_gain_at_10": 0.2234,
    "normalized_discounted_cumulative_gain_at_25": 0.2895,
    "normalized_discounted_cumulative_gain_at_5": 0.1889,
    "precision_at_10": 0.0588,
    "precision_at_25": 0.0418,
    "precision_at_5": 0.0824
  },
  "ResponseMetadata": {
    "RequestId": "662598d1-86f0-49a1-98a9-1c3924882f4b",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 00:41:11 GMT",
      "x-amzn-requestid": "662598d1-86f0-49a1-98a9-1c3924882f4b",
      "content-length": "404",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


이 예에서는 5개 항목의 정밀도가 약간 상승하여 이번에는 전체 항목의 경우 (전체 4.5%, 소규모 6.4%)를 약간 넘었습니다. 사실상 이는 오차 범위 내이지만 인기 항목을 마스킹하려는 시도를 전혀 하지 않은 점을 고려하면, 많은 수의 사용자들이 어떤 식으로든 상호 작용한 매우 인기 있는 결과를 반환하는 것일 수도 있습니다.

### 개인별 순위 지표

이제 개인별 순위 솔루션 버전에 대한 평가 지표를 검색합니다.

In [25]:
rerank_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = rerank_solution_version_arn
)

print(json.dumps(rerank_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:136455442858:solution/personalize-poc-rerank/eae37049",
  "metrics": {
    "coverage": 0.0038,
    "mean_reciprocal_rank_at_25": 0.0537,
    "normalized_discounted_cumulative_gain_at_10": 0.0707,
    "normalized_discounted_cumulative_gain_at_25": 0.0956,
    "normalized_discounted_cumulative_gain_at_5": 0.0578,
    "precision_at_10": 0.0132,
    "precision_at_25": 0.0113,
    "precision_at_5": 0.0189
  },
  "ResponseMetadata": {
    "RequestId": "01f86e05-9408-4456-97d3-2548d2cc998e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 00:41:11 GMT",
      "x-amzn-requestid": "01f86e05-9408-4456-97d3-2548d2cc998e",
      "content-length": "406",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


이에 대한 간단한 설명을 드리자면, 여기서는 사용자 개인화를 기반으로 하기 때문에 정밀도가 (전체 2.7%, 소규모 2.2%)에 가깝다는 것을 알 수 있습니다. 하지만 검증에 사용되는 샘플 항목은 동일하지 않으므로 점수가 낮습니다.

## 평가 지표 사용 <a class="anchor" id="use"></a>
[맨 위로 이동](#top)

평가 지표를 신중하게 사용하는 것이 중요합니다. 기억해야 할 여러 가지 요소가 있습니다.

* 기존 추천 시스템이 있는 경우, 새 솔루션을 훈련하는 데 사용할 사용자의 상호 작용 기록에 그 시스템이 이미 영향을 미쳤을 것입니다. 즉, 평가 지표가 기존 솔루션에 적합하도록 편향되어 있습니다. 기존 솔루션과 일치하거나 초과하도록 평가 지표를 푸시할 경우 기존 솔루션처럼 작동하도록 사용자 개인화를 푸시할 뿐 더 나은 결과는 얻어지지 않을 수 있습니다.
* HRNN Coldstart 레시피는 Amazon Personalize에서 생성된 지표를 사용하여 평가하기가 어렵습니다. 이 레시피의 목적은 비즈니스에 새로운 항목을 추천하는 것입니다. 따라서 이러한 항목은 평가 지표를 계산하는 데 사용되는 기존 사용자 트랜잭션 데이터에는 나타나지 않습니다. 결과적으로, 평가 지표와 비교했을 때 HRNN Coldstart는 전혀 다른 레시피보다 더 나은 성능을 발휘하지 못하는 것으로 보입니다. 참고: 사용자 개인화 레시피에는 향상된 콜드 스타트 기능도 포함되어 있습니다.

이러한 요인을 고려하여 Personalize에서 생성된 평가 지표는 일반적으로 다음 두 가지 경우에 유용합니다.
1. 동일한 레시피에 대해서 훈련했지만 서로 다른 하이퍼파라미터 값과 특성 값(노출 데이터 등)을 사용하는 여러 솔루션 버전의 성능을 비교합니다.
1. 서로 다른 레시피에 대해 훈련한 여러 솔루션 버전의 성능을 비교합니다(HRNN Coldstart 제외).

어떤 경우든 추천 시스템을 적절하게 평가하는 가장 좋은 방법은 A/B 테스트를 통해 실제 비즈니스 결과를 측정하는 것입니다. 시스템에 의해 생성된 추천은 일반적으로 시스템이 기반으로 하는 사용자 행동에 영향을 미치기 때문에 소규모 실험을 실행하고 장기간에 걸쳐 A/B 테스트를 적용하는 것이 좋습니다. 시간이 지남에 따라 기존 모델의 편향이 희미해집니다.

## 유용한 변수 저장 <a class="anchor" id="vars"></a>
[맨 위로 이동](#top)

이 노트북을 종료하기 전에 다음 셀을 실행하여, 다음 노트북에서 사용할 버전 ARN을 저장하세요.

In [26]:
%store userpersonalization_solution_version_arn
%store sims_solution_version_arn
%store rerank_solution_version_arn
%store user_personalization_solution_arn
%store sims_solution_arn
%store rerank_solution_arn

Stored 'userpersonalization_solution_version_arn' (str)
Stored 'sims_solution_version_arn' (str)
Stored 'rerank_solution_version_arn' (str)
Stored 'user_personalization_solution_arn' (str)
Stored 'sims_solution_arn' (str)
Stored 'rerank_solution_arn' (str)
